# SVD_Recommender System

This assignment has two main parts:

    1. **PCA** : In this part the goal is to implement the dimensionality reduction technique *Principal Component Analysis (PCA)* to a very high dimensional data and apply visualization. Note that you are not allowed to use the built-in PCA API provided by the sklearn library. Instead you will be implementing from the scratch. Use the data in data/train.csv for generating the PCA. See the detailed intructions below.
    
    2. **Recommendation system** : In this part use SVD to get USVT decomposition on the data in train.csv to recommend the movies to the users in test.csv. The submissions.csv should contain user_id (from the test.csv) followed by recommended ratings for all movies.

   For this task we use the  MovieLens dataset. The data is in train.csv.
   

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Part-1a: Convert data to user-movie rating matrix
    - Read the train.csv file and movies.dat file and use user_id and movie_id to create user-movie rating matrix


In [20]:
def readMovieRatingData():
    
    train_df = pd.read_csv('./data/train.csv')
    user_movie_df = pd.pivot_table(train_df, values='rating', index=['user_id'], columns=['movie_id'], fill_value=0)
    user_movie_matrix = user_movie_df.to_numpy()  
      
    
    return user_movie_matrix

In [3]:
def readMovieDeata():
    
    col_names = ['movie_id','title','genres']
    movies = pd.read_table('./data/movies.dat' , sep= '::' , engine='python' , header= None , names= col_names)
    movies['genres'] = movies['genres'].map(lambda x: x.split('|'))
    
    return movies

## We are going to compute PCA for movies so transpose the matrix using X=readMovieRatingData().T


# Part-1b: Preprocessing
Before implementing PCA you are required to perform some preprocessing steps:
1. Mean normalization: Replace each feature/attribute, $x_{ji}$ with $x_j - \mu_j$, In other words, determine the mean of each feature set, and then for each feature subtract the mean from the value, so we re-scale the mean to be 0 
2. Feature scaling: If features have very different scales then scale make them comparable by altering the scale, so they all have a comparable range of values e.g. $x_{ji}$ is set to $(x_j - \mu_j) / s_j$  Where $s_j$ is some measure of the range, so could be  $\max(x_j) - \min(x_j)$ or Standard deviation $stddev(x_j)$.

In [4]:
# TODO We can see features have very different scales. So we apply feature scaling with Standard 
# deviation as measure of the range, using StandardScaler from scikit-learn

def preprocessing(X_train):
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    
    return X_scaled

# Part-2: Covariance matrix
Now the preprocessing is finished. Next, as explained in the lecture, you need to compute the covariance matrix https://en.wikipedia.org/wiki/Covariance_matrix. Given $n \times m$ $n$ rows and $m$ columns matrix, a covariance matrix is an $n \times n$ matrix given as below (sigma)
$\Sigma = \frac{1}{m}\sum{\left(x^{i}\right)\times \left(x^{i}\right)^{T}}$
You may use the "numpy.cov" function in numpy library 

In [5]:
# Compute X to covariance matrix cov_matrix.
# This block is implemented in runPCA().

# Instructions for part 3, 4, and 5
- getSVD() function is expected to return 3 values. For example: ```U, S, V = getSVD(cov_matrix)```
- You can follow the skeleton below to have an idea on how the autograder's test calls your functions:
```
U, S, V = getSVD(cov_matrix)
z = getKComponents(U, X, k)
ratio = getVarianceRatio(z, U, X, k)
```
- Using the built-in PCA implementation in sklearn, the approximate X matrix can be obtained by function ```inverse_transform```

# Part-3: SVD computation
Now compute the SVD on the covariance matrix $SVD(\Sigma)$. You may use the svd implementation in numpy.linalg.svd

In [6]:
def getSVD(cov_matrix):
    U, S, V = np.linalg.svd(cov_matrix, full_matrices=False)
    return U, S, V

# Part-4: Compute PCA matrix (K dimensional)
Now select the first $k$ columns from the matrix $U$ and multiply with $X$ to get $k$ dimensional representation.

In [7]:
def getKComponents(U, X, K):
    # implement matrix multiplication of first k columns of U * X
    
    U_reduced = U[: ,: K]
    Z = X.dot(U_reduced)
    
    return Z

# Part-5: Compute Reconstruction Error
Implement a function to compute the variance ratio (from reconstruction error)

In [8]:
def getVarianceRatio(Z, U, X, K):
    #Implement computation of reconstruction error
    
    U_reduced = U[: ,: K]
    X_approx = Z.dot(U_reduced.T)
    ratio = np.mean((X-X_approx).T.dot(X-X_approx))/np.mean(X.T.dot(X))  
    
    return ratio

# Part-6: Scatter plot 2-dimensional PCA
Using matplotlib plot the 2-dimensional scatter plot of the first 2 compoenents with y (movie genre from movies.dat file) as labels. Remember you are plotting movies in dimensions so you can label them with movie generes.

In [9]:
def map_genre(x):
    types = ['Action', 'Animation', 'Comedy', 'Drama', 'Romance', 'Thriller', 'Adventure', 'Sci-Fi', 'War', 'Documentary']
    for t in types:
        if t in x:
            return t
    return 'Other'

In [10]:
def plotFunction(PCA, movie_data):
    
    train = pd.read_csv('./data/train.csv')
    movie_id_index = np.array(sorted(train['movie_id'].unique()))
    df = pd.DataFrame(data=PCA, index= movie_id_index , columns=['PC1','PC2'])
    df_new = df.merge(movie_data , how='inner' , left_index=True , right_on=['movie_id'])
    df_new['mapped_genre'] =  df_new.genres.apply(lambda x: map_genre(x))
    df_new.drop(['movie_id','title','genres'] , axis=1)
    
    fig = plt.figure(figsize = (14,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    movie_types = ['Action', 'Animation', 'Comedy', 'Drama', 'Romance', 'Thriller', 'Adventure',
               'Sci-Fi', 'War', 'Documentary' , 'Other']

    for item in movie_types:
        indices = df_new['mapped_genre'] == item
        ax.scatter(df_new.loc[indices, 'PC1'] , df_new.loc[indices, 'PC2'] , s = 50)

    ax.legend(movie_types)
    ax.grid()
    
       


# Part-7 Find best $K$
Find the minimum value of $K$ with which the ratio between averaged squared projection error with total variation in data is less than 0.1% in other words we retain 99.9% of the variance. You can achieve this by repeating getKComponents with $K=1$ until the variance ratio is <= 0.1%.

In [18]:
def findBestK(initial, step):
    
    X = readMovieRatingData().T
    X = preprocessing(X)
    cov_matrix = np.cov(X.T)
    U,_,_ = getSVD(cov_matrix)
    
    while(True):
        
        Z = getKComponents(U, X, initial)
        ratio = getVarianceRatio(Z, U, X, initial)
        
        if (ratio <= 0.001):
            break
        else:
            initial += step          
           
    return initial

# Part-8: TSNE visualization
Finally, having found an optimal $K$ use these components as an input data to another dimensionality reduction method called tSNE (https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) and reduce it to 2 dimensions.

Finally, scatter plot the components given by the tSNE using matplotlib compare it to the earlier scatter plot.

In [3]:
def plotFunction_tsne(tsne_pca_results, movie_data):
    
    train = pd.read_csv('./data/train.csv')
    movie_id_index = np.array(sorted(train['movie_id'].unique()))
    df = pd.DataFrame(data=tsne_pca_results, index= movie_id_index , columns=['PC1','PC2'])
    df_new = df.merge(movie_data , how='inner' , left_index=True , right_on=['movie_id'])
    df_new['mapped_genre'] =  df_new.genres.apply(lambda x: map_genre(x))
    df_new.drop(['movie_id','title','genres'] , axis=1)
    
    df_new['tsne-pca-one'] = tsne_pca_results[:,0]
    df_new['tsne-pca-two'] = tsne_pca_results[:,1]
    
    plt.figure(figsize=(16,10))

    sns.scatterplot(
        x="tsne-pca-one", y="tsne-pca-two",
        hue="mapped_genre",
        palette=sns.color_palette("hls", 11),
        data=df_new,
        legend="full",
        alpha=0.3,
    )

# Part-9: Recommender System
## Part-9a
    - In this part you will use the SVD to build your own recommender engine for the movielens data
    - Use the user-movie rating matrix from the training data (data/train.csv) to decmopose it into USV^T or use getSVD function from earlier
    - Convert the S to the diagonal matrix using np.diag
    - Take k best components (extract kxk matrix). k value can be using PCA k_min you found earlier
    - Take square root of S matrix using scipy.sqrtm package as s_squre_root
    - Multiply take U_reduced (first k columns of U) with s_squre_root (nxk . kxk)
    - Then multiply the result from previous step with V_reduced which is a kxm matrix and return a recommendation matrix

In [14]:
def getRecommendationMatrix(U, s, V, k):
    
    s = np.diag(s)
    s_reduced = s[0:k,0:k]
    s_reduced = sqrtm(s_reduced)
    
    U_reduced = U[:,0:k]
    V_reduced = V[0:k,:]
    result = U_reduced.dot(s_reduced)
    rec_matrix = result.dot(V_reduced)
    
    
    return rec_matrix

## Part-9a
    - Use the recommendation matrix from getRecommendationMatrix to recommend movies for the user-movie pairs in data/test.csv
    - If user-movie pair exits in the training data, use the matrix value as the recommended rating, else take the mean of the ratings for that movie and recommend that
    - Write the recommended ratings in submissions.csv 

In [15]:
def getMovieRecommendations():
    # Use user-movie rating matrix X from readMovieRatingData() earlier to compute SVD
    # Read data/test.csv in a similar way and get the test dataframe
    
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    train = pd.pivot_table(train, values='rating', index=['user_id'], columns=['movie_id'], fill_value=0)
    movie_indices = train.columns.tolist()
    
    X = readMovieRatingData()
    mask = X==0
    masked_arr = np.ma.masked_array(X, mask)
    item_means = np.mean(masked_arr, axis=0)
    X = masked_arr.filled(item_means)
    mean_matrix = np.tile(item_means, (X.shape[0],1))
    X = X - mean_matrix
    
    U,S,V = getSVD(X)
    rec_matrix = getRecommendationMatrix(U,S,V,17)
    rec_matrix += mean_matrix
    
    masked_rec = np.ma.masked_array(rec_matrix, mask)
    rec_means = np.mean(masked_rec, axis=0)
    
    
    with open('submissions.csv', 'w') as sub_file:
        sub_file.write('user_id,movie_id,rating'+ '\n')
        for _,row in test.iterrows():
            
            user = row['user_id']
            item = row['movie_id']
                        
            if (item not in movie_indices):
                pred_rating = 0
                                  
            elif (train.loc[user][item] != 0):
                idx = movie_indices.index(item)
                pred_rating = rec_matrix[int(user)-1][idx]
                
            else:
                idx = movie_indices.index(item)
                pred_rating = rec_means[idx]
                
            sub_file.write(str(user) + ',' + str(item) + ',' + str(pred_rating) + '\n')

In [16]:
def runPCA():
    X = readMovieRatingData().T
    movie_data = readMovieDeata()
    X = preprocessing(X)
    cov_matrix = np.cov(X.T)
    U, S, V = getSVD(cov_matrix)
    Z = getKComponents(U, X, 2)
    ratio = getVarianceRatio(Z, U, X, 2)
    #plotFunction(Z, movie_data)                      # uncomment for showing the plot for part 6
    K_best = findBestK(2, 2)
    
    pca = PCA(n_components=K_best)                    # TSNE Plot part8
    z_pca = pca.fit_transform(X)
    
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_pca_results = tsne.fit_transform(z_pca)
    #plotFunction_tsne(tsne_pca_results, movie_data)       # uncomment for showing the plot for part 8
    



In [21]:
if __name__ == "__main__":
    runPCA()
    getMovieRecommendations()